<a href="https://colab.research.google.com/github/Madjogger1202/Team1_AtomicHack2023/blob/main/combined_solve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import-ы

In [ ]:
!pip install rdkit
!python -m pip install git+https://github.com/EBjerrum/molvecgen

  Cloning https://github.com/EBjerrum/molvecgen to /tmp/pip-req-build-i4u5ntwo
  Running command git clone --filter=blob:none --quiet https://github.com/EBjerrum/molvecgen /tmp/pip-req-build-i4u5ntwo
  Resolved https://github.com/EBjerrum/molvecgen to commit f81d5aade18bea60882f5845877f6283366bbe91
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Draw, Descriptors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from molvecgen.vectorizers import SmilesVectorizer
from pprint import pprint
from sklearn.preprocessing import FunctionTransformer

Предобработка данных: разложить формулу, добавить векторизированные параметры

In [ ]:
data = pd.read_excel('35000.xlsx') #вводим данные

In [ ]:
def mol_dsc_calc(mols):
    return pd.DataFrame({k: f(Chem.MolFromSmiles(m)) for k, f in descriptors.items()} for m in mols)

# список конституционных и физико-химических дескрипторов из библиотеки RDKit
descriptors = {"HeavyAtomCount": Descriptors.HeavyAtomCount,
               "NHOHCount": Descriptors.NHOHCount,
               "NOCount": Descriptors.NOCount,
               "NumHAcceptors": Descriptors.NumHAcceptors,
               "NumHDonors": Descriptors.NumHDonors,
               "NumHeteroatoms": Descriptors.NumHeteroatoms,
               "NumRotatableBonds": Descriptors.NumRotatableBonds,
               "NumValenceElectrons": Descriptors.NumValenceElectrons,
               "NumAromaticRings": Descriptors.NumAromaticRings,
               "NumAliphaticHeterocycles": Descriptors.NumAliphaticHeterocycles,
               "RingCount": Descriptors.RingCount,
               "MW": Descriptors.MolWt,
               "LogP": Descriptors.MolLogP,
               "MR": Descriptors.MolMR,
               "TPSA": Descriptors.TPSA,
               }

# sklearn трансформер для использования в конвейерном моделировании
descriptors_transformer = FunctionTransformer(mol_dsc_calc)
X = descriptors_transformer.transform(data['SMILES'])
X.head()

[05:05:05] Conflicting single bond directions around double bond at index 55.
[05:05:05]   BondStereo set to STEREONONE and single bond directions set to NONE.
[05:05:05] Conflicting single bond directions around double bond at index 55.
[05:05:05]   BondStereo set to STEREONONE and single bond directions set to NONE.
[05:05:05] Conflicting single bond directions around double bond at index 55.
[05:05:05]   BondStereo set to STEREONONE and single bond directions set to NONE.
[05:05:05] Conflicting single bond directions around double bond at index 55.
[05:05:05]   BondStereo set to STEREONONE and single bond directions set to NONE.
[05:05:05] Conflicting single bond directions around double bond at index 55.
[05:05:05]   BondStereo set to STEREONONE and single bond directions set to NONE.
[05:05:05] Conflicting single bond directions around double bond at index 55.
[05:05:05]   BondStereo set to STEREONONE and single bond directions set to NONE.
[05:05:05] Conflicting single bond direc

,HeavyAtomCount,NHOHCount,NOCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumAliphaticHeterocycles,RingCount,MW,LogP,MR,TPSA
0,24,7,9,4,6,10,7,138,0,0,1,362.367,0.38187,90.4296,157.76
1,21,4,6,4,3,6,7,120,0,0,1,298.383,1.05450,79.7279,101.65
2,21,3,6,5,2,6,7,118,0,0,1,295.359,-0.36170,77.0531,104.48
3,20,3,6,5,2,6,6,114,0,0,1,283.348,-0.52780,72.5301,104.48
4,24,4,9,6,4,9,3,122,3,0,3,329.268,1.09742,81.9775,149.28


In [ ]:
Y = data['IC50'].values.reshape(-1, 1)
print(Y.shape)
X_arr = np.reshape(X.values.flatten().tolist(), (36377, 15))
print(X.shape)
print(X_arr[0])

(36377, 1)
(36377, 15)
[ 24.        7.        9.        4.        6.       10.        7.
 138.        0.        0.        1.      362.367     0.38187  90.4296
 157.76   ]


In [ ]:
data.describe()

,Unnamed: 0,IC50
count,0.0,36377.000000
mean,NaN,7.813503
std,NaN,48.489710
min,NaN,0.000010
25%,NaN,1.696000
50%,NaN,3.635000
75%,NaN,5.934000
max,NaN,3183.600000


In [ ]:
class bigDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return self.x[idx], self.y[idx]

train_dataset=bigDataset(X_arr, Y)

#Первая попытка

In [ ]:
X_data = X_arr

Y_data = Y
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data,  test_size=0.25, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train,  test_size=0.2, random_state=42)
#Normalizing output using standard scaling
scaler = StandardScaler()
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)
y_validation = scaler.transform(y_validation)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
X_validation_t = X_validation#smivec.transform(X_validation, canonical=False)
X_validation_t = torch.tensor(X_validation_t, device=device).float()
y_validation_t = torch.tensor(y_validation, device=device).float()
y_validation_t = torch.nan_to_num(y_validation_t, nan = torch.nanmean(y_validation_t))
X_validation_t.shape



class Net(nn.Module):
    def __init__(self, dimensions, lstm_size, hidden_size, dropout_rate, out_size):
        super(Net, self).__init__()

        length = 36377
        number_tokens = 15


        self.lstm = nn.GRU(input_size=number_tokens, hidden_size=lstm_size, num_layers=1, batch_first=True, bidirectional=False, dropout  = 0.2)
        self.fc1 = nn.Linear(lstm_size, hidden_size) # Output layer
        self.activation = nn.ReLU() # Non-Linear ReLU Layer
        self.fc2 = nn.Linear(hidden_size, hidden_size//2) # Output layer
        self.activation2 = nn.ReLU() # Non-Linear ReLU Layer
        self.fc_out = nn.Linear(hidden_size//2, out_size) # Output layer
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):# Forward pass: stacking each layer together
        #out, (h_n, c_n) = self.lstm(x) #LSTM network reads in one-hot-encoded SMILES, h_n is last output, out is for all timesteps
        out, h_n = self.lstm(x) #GRU network reads in one-hot-encoded SMILES, h_n is last output, out is for all timesteps
        out = self.dropout(h_n) #Dropout
        out = self.fc1(out) # Pass into the hidden layer
        out = self.activation(out) # Use ReLU on hidden activation
        out = self.fc2(out) # Pass into the hidden layer
        out = self.activation2(out) # Use ReLU on hidden activation
        out = self.dropout(out) # dropout
        out = self.fc_out(out) # Use a linear layer for the output
        return out



epochs = 100
lstm_size = 15  # The size of the LSTM layer
hidden_size = 500   # The size of the hidden non-linear layer
dropout_rate = 0.1 # The dropout rate
output_size = 1        # This is just a single task, so this will be one
batch_size = 64   # The mini_batch size during training
learning_rate = 0.0005  # The initial learning rate for the optimizer

model = Net((36377, 15), lstm_size, hidden_size, dropout_rate, output_size)
model.cpu()

pred = model.forward(X_validation_t[10:11])
pred

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=50,
                  verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=1e-6, eps=1e-08)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True, num_workers=4)




for smiles, labels in train_loader:
    print(smiles)
    print(labels)
    break



model.train() #Ensure the network is in "train" mode with dropouts active
train_losses = []
validation_losses = []
for e in range(epochs):
    running_loss = 0
    for smiles, labels in train_loader:
        # Push numpy to CUDA tensors
        smiles = torch.tensor(smiles, device=device).float()
        #print(smiles)
        labels = torch.tensor(labels, device=device).float()
        labels = torch.nan_to_num(labels, nan = torch.nanmean(labels))
        #print(labels)
        #break
        # Training pass
        optimizer.zero_grad() # Initialize the gradients, which will be recorded during the forward pass

        output = model(smiles) #Forward pass of the mini-batch
        loss = criterion(output, labels) #Computing the loss
        loss.backward() # calculate the backward pass
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step() # Optimize the weights

        running_loss += loss.item()
    else:


        model.eval()
        validation_loss = torch.mean(( y_validation_t - model(X_validation_t) )**2).item()
        model.train()

        lr_scheduler.step(validation_loss)

        train_loss = running_loss/len(train_loader)
        train_losses.append(train_loss)
        validation_losses.append(validation_loss)
        if (e+1)%5 == 0:
            print("Epoch %i, Training loss: %0.2F Validation loss: %0.2F"%(e + 1, train_loss, validation_loss))


#Вторая попытка

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn import datasets


inputs = keras.Input(shape=(15,), name='digits')
x = layers.Dense(200, activation='relu', name='dense_1')(inputs)
x = layers.Dropout(0.1)(x)
x = layers.Dense(200, activation='relu', name='dense_2')(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(	3900, activation='softmax', name='predictions')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

x_train=X_arr
y_train=Y.reshape(-1, 1)
#(36377, 1)
#(36377, 15)
# Предобработаем данные (это массивы Numpy)
x_train = X_arr.astype('float32')
x_test1 = Y.reshape(-1, 1).astype('float32')

x_test=x_train
y_test=y_train

# Укажем конфигурацию обучения (оптимизатор, функция потерь, метрики)
model.compile(optimizer=keras.optimizers.RMSprop(),  # Optimizer
              # Минимизируемая функция потерь
              loss=keras.losses.SparseCategoricalCrossentropy(),
              # Список метрик для мониторинга
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

# Обучим модель разбив данные на "пакеты"
# размером "batch_size", и последовательно итерируя
# весь датасет заданное количество "эпох"
print('# Обучаем модель на тестовых данных')
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=50,
                    # Мы передаем валидационные данные для
                    # мониторинга потерь и метрик на этих данных
                    # в конце каждой эпохи
                    validation_data=(x_test, y_test))

# Возвращаемый объект "history" содержит записи
# значений потерь и метрик во время обучения
print('\nhistory dict:', history.history)

# Оценим модель на тестовых данных, используя "evaluate"
print('\n# Оцениваем на тестовых данных')
results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc:', results)
model.save('model.h5')
# Сгенерируем прогнозы (вероятности - выходные данные последнего слоя)
# на новых данных с помощью "predict"
print('\n# Генерируем прогнозы для 3 образцов')
predictions = model.predict(x_test[:3])
print('размерность прогнозов:', predictions.shape)
print(predictions)

# Обучаем модель на тестовых данных
Epoch 1/50
569/569 [==============================] - 17s 29ms/step - loss: 3.0166 - sparse_categorical_accuracy: 5.4980e-04 - val_loss: 2.6717 - val_sparse_categorical_accuracy: 1.3745e-04
Epoch 2/50
569/569 [==============================] - 16s 28ms/step - loss: 2.7156 - sparse_categorical_accuracy: 4.3984e-04 - val_loss: 2.6265 - val_sparse_categorical_accuracy: 2.7490e-04
Epoch 3/50
569/569 [==============================] - 17s 31ms/step - loss: 2.6760 - sparse_categorical_accuracy: 5.4980e-04 - val_loss: 2.6164 - val_sparse_categorical_accuracy: 6.8725e-04
Epoch 4/50
569/569 [==============================] - 16s 28ms/step - loss: 2.6454 - sparse_categorical_accuracy: 4.3984e-04 - val_loss: 2.6609 - val_sparse_categorical_accuracy: 6.8725e-04
Epoch 5/50
569/569 [==============================] - 16s 28ms/step - loss: 2.6285 - sparse_categorical_accuracy: 4.9482e-04 - val_loss: 2.6315 - val_sparse_categorical_accuracy: 6.5976e-04
Epoch 6/50
569

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [76]:
model_loaded = keras.models.load_model('model.h5')
predictions = (model_loaded.predict((x_test[36360:36361])))
print("Ожидаемый результат: ")
print(Y[36360])
print('размерность прогнозов:', predictions.shape)
#predictions.sort()
print("Вероятности: ")
print(predictions)
print("Ответ модели IC50: ")
print(max(range(len(predictions[0])), key=lambda i: predictions[0][i]))
print("Вероятность для искомого ответа:")
print(predictions[0][1000])

1/1 [==============================] - 0s 61ms/step
Ожидаемый результат: 
[1000.]
размерность прогнозов: (1, 3900)
Вероятности: 
[[4.1668209e-01 1.6152768e-01 2.1563546e-01 ... 3.2958758e-06
  3.1098189e-06 3.4381612e-06]]
Ответ модели IC50: 
0
Вероятность для искомого ответа:
8.370878e-06


In [ ]:
"HeavyAtomCount"
"NHOHCount"
"NOCount"
"NumHAcceptors"
"NumHDonors"
"NumHeteroatoms"
"NumRotatableBonds"
"NumValenceElectrons"
"NumAromaticRings"
"NumAliphaticHeterocycles"
"RingCount"
"MW"
"LogP"
"MR"
"TPSA"
